# Parameter efficient fine-tuning

В рамках семинаров мы ограничены одной не очень большой GPU доступной на колабе (t4 с 16 гб памяти). Поэтому в предыдущих семинарах, когда нужно было что-то зафайнтюнить мы использовали самые маленькие языковые модели (opt-125m, например), иначе мы бы столкнулись с OOM ошибкой или слишком долгим обучением. Естественно качество таких моделей не впечатляет и хотелось бы попробовать модели побольше. Даже сильно побольше, так как кажется, что [эмержентные](https://ru.wikipedia.org/wiki/%D0%AD%D0%BC%D0%B5%D1%80%D0%B4%D0%B6%D0%B5%D0%BD%D1%82%D0%BD%D0%BE%D1%81%D1%82%D1%8C)
свойства, о которых все сейчас говорят, начинают проявлятся у моделей размером около 6-10 миллиардов параметров (https://arxiv.org/pdf/2206.07682.pdf).
По умолчанию модель opt-6.7b требует около 25 гб видеопамяти, то есть даже для инференса ресурсов колаба не хватит, не говоря даже о обучении (для него понадобится в 4 раза больше).

К счастью нехватка ресурсов - общая проблема. Даже те, у кого есть такие ресурсы заинтересованы в оптимизации (можно использовать меньше ресурсов=денег или же использовать такое же количество ресурсов, но обслуживать больше пользователей=зарабатывать больше денег). Поэтому усилия многих исследователей и компаний направлены в сторону оптимизации больших языковых моделей.


В этом семинаре мы разберем несколько уже разработанных подходов и сможем обучить модель ~~facebook/opt-6.7b в колабе~~ (на самом деле получится только opt-1.3b, так как все библиотеки еще новые и нестабильные)!


Оптимизация базовой модели и оптимизация процесса дообучения это немного разные вещи, поэтому разберем их по очереди.



## Оптимизация предобученных моделей
Для уменьшения готовой модели есть два основных подхода: дистиляция и квантизация.

**Дистиляция** (knowledge distillation) - это обучение меньшей по размеру модели воспроизводить предсказания большей модели. Исходная модель при таком подходе называется учителем, а меньшая модель - учеником. Например, можно взять большой предобученный классификатор тональности, сделать предсказания на каком-нибудь большом корпусе и обучить меньшую модель на этих предсказаниях. Так как при уменьшении количества параметров будет уменьшаться и качество, можно ограничить обучающий корпус каким-то одним доменом, чтобы упростить меньшей модели задачу (меньшая модель, например, научится хорошо определять тональность коротких текстов, но будет плохо работать с длинными; на практике это может быть приемлимый трейдофф)

Схема дистилляции:
![](https://miro.medium.com/v2/resize:fit:936/1*8KqNtABnNXM527JK9UuBUQ.jpeg)

Применять дистиляцию к простым языковым моделям нет особого смысла, так как обучающие данные и так доступны и можно просто обучить меньшую модель с нуля аналогично большой модели. Но уместна дистиляция для моделей, обученных на инструкциях. Вот например эксперименты по дистиляции GPT в небольшие модели - https://github.com/mbzuai-nlp/LaMini-LM В целом это очень похоже на [Alpaca](https://github.com/tatsu-lab/stanford_alpaca), но датасет здесь намного больше, а сама модель обучается с нуля (в Alpaca дообучается Llama). И также как и с Alpaca, это не очень то легально - по сути это попытка скопировать модель без доступа к обучающим данным, поэтому лицензия на моделях запрещает коммерческое использование.


**Квантизация** - это уменьшение размера модели за счет уменьшения точности представления чисел. Веса в модели это просто очень много чисел вида 0.23123125, -1.234559 и для хранения каждого такого числа требуется какое-то количество памяти. Интервал допустимых значений и количество знаков после запятой всегда ограничены, но по умолчанию они достаточно большие и, выясняется, что можно достаточно сильно округлить веса, и при этом, практически не потерять в качестве!
Схема квантизации:
![](https://developer-blogs.nvidia.com/wp-content/uploads/2021/07/qat-training-precision.png)

Разумеется, квантизация сложнее, чем просто округление, но подробно методы квантизации мы разбирать не будем. Если вам интересно, можно почитать вот это - https://huggingface.co/blog/hf-bitsandbytes-integration Один из авторов - Tim Dettmers, автор библиотеки bitsandbytes, в которой реализованы методы квантизации и которая постепенно интегрируется в huggingface.

Давайте посмотрим как использовать квантизацию в huggingface с предобученными моделями.

In [1]:
# !pip install -q git+https://github.com/huggingface/transformers.git@main
# !pip install accelerate

Попробуем загрузить большую модель без дополнительных параметров.

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [3]:
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path='facebook/opt-2.7b',
                                             cache_dir='./models').to('cuda')
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-2.7b")

/home/ubuntu/.pyenv/versions/3.10.14/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
batch = tokenizer("In the beginning the Universe was created.", return_tensors='pt').to('cuda')
output_tokens = model.generate(**batch, max_new_tokens=50, temperature=0.1, do_sample=True, no_repeat_ngram_size=2)
print(tokenizer.decode(output_tokens[0], skip_special_tokens=True))

/home/ubuntu/.pyenv/versions/3.10.14/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(




 In the beginning the Universe was created. This has made a lot of people very angry and been widely regarded as a bad move.
I'm not sure if I should upvote or downvote this.


Можно посмотреть сколько такая модель занимает памяти на gpu

In [5]:
!nvidia-smi

Thu Apr 18 12:27:27 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A10G         On   | 00000000:00:1E.0 Off |                    0 |
|  0%   24C    P0   129W / 300W |  10635MiB / 23028MiB |     79%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


По умолчанию веса хранятся в fp32 и модель занимает 10гб на gpu. Модель opt6.7B при таком формате не поместится в память.

In [7]:
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path='facebook/opt-6.7b',
                                             cache_dir='./models').to('cuda')

config.json:   0%|          | 0.00/651 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/41.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.36G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 256.00 MiB (GPU 0; 22.06 GiB total capacity; 21.56 GiB already allocated; 190.38 MiB free; 21.58 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

### Не забудьте перезагрузить кернел после OOM ошибки!

## FP16

Давайте попробуем загрузить модель в fp16 (из названия можно догадаться, что этот формат в два раза меньше)

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [2]:
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path='facebook/opt-2.7b',
                                             torch_dtype=torch.float16, # указываем fp16
                                             cache_dir='./models').to('cuda')
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-2.7b")

/home/ubuntu/.pyenv/versions/3.10.14/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [3]:
batch = tokenizer("In the beginning the Universe was created.", return_tensors='pt').to('cuda')
output_tokens = model.generate(**batch, max_new_tokens=50, temperature=0.1, do_sample=True, no_repeat_ngram_size=2)
print(tokenizer.decode(output_tokens[0], skip_special_tokens=True))

/home/ubuntu/.pyenv/versions/3.10.14/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(




 In the beginning the Universe was created. This has made a lot of people very angry and been widely regarded as a bad move.
I'm not sure if I should upvote or downvote this.


Теперь занято около 5гб!

In [4]:
!nvidia-smi

Thu Apr 18 12:34:21 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A10G         On   | 00000000:00:1E.0 Off |                    0 |
|  0%   24C    P0    70W / 300W |   5613MiB / 23028MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Давайте попробуем модель с 6.7B параметрами еще раз

In [5]:
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path='facebook/opt-6.7b',
                                             torch_dtype=torch.float16, # указываем fp16
                                             cache_dir='./models').to('cuda')
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-6.7b")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/651 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

In [6]:
batch = tokenizer("In the beginning the Universe was created.", return_tensors='pt').to('cuda')
output_tokens = model.generate(**batch, max_new_tokens=50, temperature=0.1, do_sample=True, no_repeat_ngram_size=2)
print(tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 In the beginning the Universe was created. This has made a lot of people very angry and been widely regarded as a bad move.
I'm not sure if you're referencing the Big Bang Theory, but I'm pretty sure that's not how it went.


#### Работает! Если посмотреть на память, то увидим 18ГБ

In [7]:
!nvidia-smi

Thu Apr 18 12:35:39 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A10G         On   | 00000000:00:1E.0 Off |                    0 |
|  0%   24C    P0    55W / 300W |  18351MiB / 23028MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## 8-bit

Можно пойти еще дальше и попробовать 8-битный формат, который требует в ~3 раза меньше памяти. Так мы можем попробовать даже еще большую модель

In [8]:
# такие квантизации работают через дополнительную библиотеку bitsandbytes
# !pip install bitsandbytes

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

In [5]:
quantization_config = BitsAndBytesConfig(
        load_in_8bit=True
    )

In [6]:
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path='facebook/opt-6.7b',
                                             quantization_config=quantization_config,
                                             device_map='auto',
                                             cache_dir='./models')
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-6.7b")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/ubuntu/.pyenv/versions/3.10.14/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [9]:
batch = tokenizer("In the beginning the Universe was created.", return_tensors='pt').to('cuda')
output_tokens = model.generate(**batch, max_new_tokens=50, temperature=0.1, do_sample=True, no_repeat_ngram_size=2)
print(tokenizer.decode(output_tokens[0], skip_special_tokens=True))

In the beginning the Universe was created. This has made a lot of people very angry and been widely regarded as a bad move.
I'm not sure if you're quoting something or not, but I like it.


In [12]:
# память снизилась даже немного больше чем в два раза
!nvidia-smi

Thu Apr 18 12:40:17 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A10G         On   | 00000000:00:1E.0 Off |                    0 |
|  0%   24C    P0    55W / 300W |   7753MiB / 23028MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## 4 bit
Можно пойти и еще дальше и сделать 4bit квантизацию! Но тут уже все становится сложнее и память снижается не так сильно. Вычисления при таком формате делаются в float16/32 формате и это тоже можно регулировать параметром

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

In [7]:
quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        # torch.float16 сократит потребление памяти, но даже так модель помещается потому что веса 4битные
        bnb_4bit_compute_dtype=torch.float32, 
        # есть еще дополнительные advanced параметры которые могут снижать потребление памяти
        # про них можно почитать тут - https://github.com/huggingface/blog/blob/main/4bit-transformers-bitsandbytes.md
        bnb_4bit_use_double_quant=True
    )

In [8]:
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path='facebook/opt-6.7b',
                                             quantization_config=quantization_config,
                                             device_map='auto',
                                             cache_dir='./models')
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-6.7b")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
batch = tokenizer("In the beginning the Universe was created.", return_tensors='pt').to('cuda')
output_tokens = model.generate(**batch, max_new_tokens=50, temperature=0.1, do_sample=True, no_repeat_ngram_size=2)
print(tokenizer.decode(output_tokens[0], skip_special_tokens=True))

/home/ubuntu/.pyenv/versions/3.10.14/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


In the beginning the Universe was created. This has made a lot of people very angry and been widely regarded as a bad move.
I'm not sure if you're referencing the movie or the book, but I'm going to assume the latter.


In [10]:
!nvidia-smi

Thu Apr 18 12:54:52 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A10G         On   | 00000000:00:1E.0 Off |                    0 |
|  0%   27C    P0    96W / 300W |   9241MiB / 23028MiB |     29%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Оптимизация обучения

Квантизация также применима при обучении моделей, но ограничено. Как правило обучают в mixed формате, когда используется float16 и float32. 8bit и 4bit пока не подходят для обучения. 


Согласно вычислениями вот отсюда - https://blog.eleuther.ai/transformer-math/ - обучение требует в ~4 раза больше памяти, чем inferece и даже если мы можем загрузить 6.7B в float16 не значит, что мы сможем ее обучать. 
Даже с квантизированной моделью полный файн-тюнинг - это все еще очень дорого. 

Но у нас есть еще одна опция, которая позволяет пользоваться и 8bit и 4bit квантизацией при обучении! Эта опация - адаптеры. 
Самый популярный на данный момент - LoRA ( https://arxiv.org/abs/2106.09685 ). Мы уже использовали частичное дообучение на предыдущих занятиях, когда добавляли 1 дополнительный полносвязный слой к предобученной модели и обучали только его. LoRA - это обобщение такого подхода, но тут, дополнительные веса можно добавить к каждому полносвязному слою в предобученной модели. 

На одном из прошлых семинаров мы разбирали, что большую матрицу можно представить как произведение нескольких матриц поменьше. Причем внутреннюю размерность этих матриц можно варьировать - чем меньше, тем хуже восстанавливается оригинальная матрица, но тем меньше памяти занимается. В LoRA используется такой же прием - к каждой матрице с весами (полносвязному слою) прибавляется матрица такого же размера, которая получена произведением двух матриц поменьше A и B. Внутренюю размерность этих матриц можно варьировать - это гиперпараметр модели (см. ниже), чем она больше, тем выше точность модели, и тем выше потребность в памяти. Такой трюк к тому же обоснован - авторы LoRA проанализировали веса в предобученных нейронных сетях и заметили, что большинство из них имеют низкий ранг! То есть их можно пред

Схема LoRA:
![](https://miro.medium.com/v2/resize:fit:730/1*D_i25E9dTd_5HMa45zITSg.png)


Веса основной модели замораживаются. При небольшой внутренней размерности получается, что дообучение адаптера требует лишь небольшой процент ресурсов, требуемых для полноценного обучения. И так как для основной модели не требуется расчитывать градиенты и обновлять веса, ее можно загружать в 8/4bit формате!

Также полученные веса очень удобно использовать на практике - их можно сохранять и подгружать отдельно, не затрагивая оригинальною большую модель. Предобученные модели весят гигабайты, а LoRA веса - несколько мегабайт. Можно также дообучить несколько LoRA весов и переключаться между ними на лету.

LoRA реализована в отдельной библиотеке PEFT в экосистеме huggingface. Давайте попробуем дообучить большую модель

### Fine-tuning

In [1]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

99.37s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
121.21s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, BitsAndBytesConfig

In [2]:
quantization_config = BitsAndBytesConfig(
        load_in_8bit=True
    )

В итоге я использую модель 2.7b модель как пример, но 6.7B (и потенциально большие) модели тоже работают, просто обучаются дольше

In [3]:
model = AutoModelForCausalLM.from_pretrained(
    "facebook/opt-2.7b", 
    quantization_config=quantization_config,
    cache_dir='./models'
)

`low_cpu_mem_usage` was None, now set to True since model is quantized.
/home/ubuntu/.pyenv/versions/3.10.14/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-2.7b")

В этой ячейке все веса изначальной модели замораживаются

In [5]:
for param in model.parameters():
  param.requires_grad = False  
  if param.ndim == 1:
    # в layernorm нужны очень маленькие числа, поэтому для него оставляют fp32 
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()
model.enable_input_require_grads()

In [6]:
# вспомогательная функция которая покажет сколько параметров будут обучаться
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

Адаптеры можно добавлять к полносвязным/линейным/dense слоям. В зависимости от архитектуры эти слои могут называться по-разному, поэтому их нужно указать вручную через парметр `target_modules`. Так как модели это в основном наслоенные однотипные трансформерные блоки, то перечислить нужно лишь несколько имен типовых слоев. 

In [7]:
# вот так можно (в torch) напечатать слои и их названия
# в нашем случае на них прямо написано, что они линейные (proj и fc)
# поэтому мы можем выбрать q_proj, v_proj, k_proj, out_proj, fc1 и fc2
for name, module in model.named_modules():
    print(name)


model
model.decoder
model.decoder.embed_tokens
model.decoder.embed_positions
model.decoder.final_layer_norm
model.decoder.layers
model.decoder.layers.0
model.decoder.layers.0.self_attn
model.decoder.layers.0.self_attn.k_proj
model.decoder.layers.0.self_attn.v_proj
model.decoder.layers.0.self_attn.q_proj
model.decoder.layers.0.self_attn.out_proj
model.decoder.layers.0.activation_fn
model.decoder.layers.0.self_attn_layer_norm
model.decoder.layers.0.fc1
model.decoder.layers.0.fc2
model.decoder.layers.0.final_layer_norm
model.decoder.layers.1
model.decoder.layers.1.self_attn
model.decoder.layers.1.self_attn.k_proj
model.decoder.layers.1.self_attn.v_proj
model.decoder.layers.1.self_attn.q_proj
model.decoder.layers.1.self_attn.out_proj
model.decoder.layers.1.activation_fn
model.decoder.layers.1.self_attn_layer_norm
model.decoder.layers.1.fc1
model.decoder.layers.1.fc2
model.decoder.layers.1.final_layer_norm
model.decoder.layers.2
model.decoder.layers.2.self_attn
model.decoder.layers.2.self_

In [9]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=32, # внутренняя размерность адаптера, основной параметр
    target_modules=["q_proj", "k_proj", "v_proj", 'out_proj', 'fc1', 'fc2'], # к каким слоям добавлять адаптеры (подробнее выше)

    # "вес" адаптера, этот параметр делится на r, то есть если они равны то
    # вес адаптера = 1 (то есть базовая модель и адаптер одинаковы по значимости)
    # если поставить этот параметр выше, то адаптер будет сильнее влиять на базовую модель
    # как я понимаю никто особо не понимает что делать с этим параметром при обучении
    # лучше оставлять его равным r
    lora_alpha=32, 
    
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 47185920 || all params: 2698782720 || trainable%: 1.7484149298243616


Параметр r отвечает за внутренюю размерность дополнительных матриц. При r=32 обучаться в итоге будет около 15м параметров, что меньше 1 процента от всех параметров изначальное модели.

### Обучение

In [10]:
import transformers
from datasets import load_dataset

# в качестве датасета я взял инструкции к генерации кода
data = load_dataset("flytech/python-codes-25k", split='train[:20%]')
data = data.map(lambda samples: tokenizer(samples['text']), batched=True)


In [11]:

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=400,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False, )
)

max_steps is given, it will override any value given in num_train_epochs


In [12]:
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

/home/ubuntu/.pyenv/versions/3.10.14/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
1,1.994500
2,2.153500
3,2.328900
4,2.536100
5,1.879700
6,2.254700
7,2.230400
8,2.497900
9,2.338700
10,2.571200


KeyboardInterrupt: 

Сохраним ~~модель~~ адаптер (сохранятся только дополнительные веса)

In [18]:
model.save_pretrained('opt_2.7_lora')

Чтобы загрузить обученный адаптер нужно сначала загрузить базовую модель, а потом применить к ней LoRa веса

In [18]:
# перед запуском этой ячейки нужно перезапустить кернел
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

peft_model_id = "opt_2.7_lora"

quantization_config = BitsAndBytesConfig(
        load_in_8bit=True
    )
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path="facebook/opt-2.7b",
                                             return_dict=True, 
                                             quantization_config=quantization_config,
                                             device_map='auto'
                                            )
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-2.7b")

In [22]:
def generate(text, tokenizer, model):
    batch = tokenizer(text, return_tensors='pt').to('cuda')
    output_tokens = model.generate(**batch, max_new_tokens=50, temperature=0.1, do_sample=True, no_repeat_ngram_size=3)

    return tokenizer.decode(output_tokens[0], skip_special_tokens=True)

Давайте попробуем сделать несколько предсказаний, используя базовую модель, чтобы потом сравнить с дообученной

In [23]:
print(generate("Write a function to see time. ",  tokenizer, model))

Write a function to see time.    If you want to see the time in seconds, you can use the time.time() function.  If the time is in minutes, you need to use the function time.minutes()
I'm not sure what you mean by


In [24]:
print(generate("Help me with a todo list! ",  tokenizer, model))

Help me with a todo list!   I have a list of things I want to do, but I don't know how to prioritize them.    What are some good resources for prioritizing tasks?
I use Trello. It's a great tool for prioritization.


Загрузим LoRA веса и попробуем те же промпты

In [25]:
model = PeftModel.from_pretrained(model, peft_model_id)


In [26]:
print(generate("Write a function to see time. ",  tokenizer, model))

Write a function to see time. 
import time

def get_time(time):
    return time.time()

time = time.now()
print(get_time('2019-01-01'))

# Output: 2019-


In [27]:
print(generate("Help me with a todo list! ",  tokenizer, model))

Help me with a todo list! 
import requests

# Create a to-do list
todo = requests.get('https://todo.io/todo/todos', 'todo', 'list')

print(todo)




Теперь модель генерирует код!